<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-3/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import tensorflow as tf
import keras as ks
import random
import csv

from tensorflow.keras.layers import Dense, Input,InputLayer,Flatten,Activation,LSTM,SimpleRNN,GRU,TimeDistributed,Embedding
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [3]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

In [4]:
!nvidia-smi

Thu Apr 28 08:28:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    34W / 250W |    375MiB / 16280MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 14.4 MB/s 
     |████████████████████████████████| 181 kB 72.1 MB/s 
     |████████████████████████████████| 144 kB 74.8 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=93a1f3119f80ff738421db323931e357a885f89302ca53c5f58c1d82e786ee23
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# English as source language and Telugu as target language
source_language='en'
target_language='te'

In [8]:
def load_data():
    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    

    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    train_tsv_file = open(train_file_path, "r", encoding="utf-8")
    train_dataset = csv.reader(train_tsv_file, delimiter="\t")

    val_tsv_file = open(val_file_path, "r", encoding="utf-8")
    val_dataset = csv.reader(val_tsv_file, delimiter="\t")

    return train_dataset, val_dataset


In [9]:
def prepare_data():

    train_dataset, val_dataset = load_data()
    
    input = []
    target = []
    #print(english)
    for i in train_dataset:
        #print(i) 
        target.append(i[0])
        #print(i[1])
        input.append(i[1])
    #print(english)
    target = np.array(target)
    input = np.array(input)

    # Validation data
    val_input = []
    val_target = []

    for i in val_dataset:
        val_target.append(i[0])
        val_input.append(i[1])

    val_target = np.array(val_target)
    val_input = np.array(val_input)

    for i in range(len(target)):
        target[i] = "\t" + target[i] + "\n"
    
    for i in range(len(val_target)):
        val_target[i] = "\t" + val_target[i] + "\n"

    return input, target, val_input, val_target



In [10]:
def getTokens(input, target, val_input, val_target):
    # Getting input and target language characters

    # Training set
    input_tokens = set()
    target_tokens = set()

    for word in input:
        for char in word:
            if char not in input_tokens:
                input_tokens.add(char)

    for word in target:
        for char in word:
            if char not in target_tokens:
                target_tokens.add(char)

    # Validation set
    val_input_tokens = set()
    val_target_tokens = set()

    for word in val_input:
        for char in word:
            if char not in val_input_tokens:
                val_input_tokens.add(char)

    for word in val_target:
        for char in word:
            if char not in val_target_tokens:
                val_target_tokens.add(char)

    input_tokens = sorted(list(input_tokens))
    target_tokens = sorted(list(target_tokens))
    
    return input_tokens, target_tokens, val_input_tokens, val_target_tokens


In [11]:
input, target, val_input, val_target = prepare_data()
input_tokens, target_tokens, val_input_tokens, val_target_tokens = getTokens(input, target, val_input, val_target)


In [12]:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(txt) for txt in input])
max_decoder_seq_length = max([len(txt) for txt in target])

input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

print("Number of data points:", len(input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of data points: 58550
Number of unique input tokens: 26
Number of unique output tokens: 65
Max sequence length for inputs: 25
Max sequence length for outputs: 20


In [13]:
print(target[5])

	అంకితభావం



ENCODER AND DECODER INPUTS

In [14]:
def one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens):

    input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])

    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    
    encoder_input_data = np.zeros(
        (len(input), max_encoder_seq_length), dtype = "float32" )
    decoder_input_data = np.zeros(
        (len(input), max_decoder_seq_length), dtype = "float32" )
    decoder_target_data = np.zeros(
        (len(input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )
    
    for i, (inp, tar) in enumerate(zip(input, target)):
        for t, char in enumerate(inp):
            encoder_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0

    

    encoder_val_input_data = np.zeros(
        (len(val_input), max_encoder_seq_length), dtype = "float32" )
    decoder_val_input_data = np.zeros(
        (len(val_input), max_decoder_seq_length), dtype = "float32" )
    decoder_val_target_data = np.zeros(
        (len(val_input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )

    for i, (inp, tar) in enumerate(zip(val_input, val_target)):
        for t, char in enumerate(inp):
            encoder_val_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_val_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0

    return encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data


In [15]:
encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data = one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens)

In [16]:
decoder_target_data.shape

(58550, 20, 65)

In [17]:
print(decoder_target_data[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
def rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # enc_in : Encoder input
  # enc_out : Encoder output
  # enc_states: Encoder states
  # dec_in : Decoder input
  # dec_out : Decoder output
  # dec_dense : Dense layer for decoder

  #=============================================#
  #============== Encoder part =================#
  #=============================================#
  
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []
  
  for i in range(num_encoders):
        encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h = encoder(enc_out)
        enc_states.append([state_h])

#   # First encoder layer of the rnn
#   encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#   enc_layers.append(encoder)
#   enc_out, state_h = encoder(enc_out)
#   enc_states.append([state_h])
  
#   # Second encoder layer of the rnn
#   if num_encoders > 1:
#     encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     enc_layers.append(encoder)
#     enc_out, state_h2 = encoder(enc_out)
#     enc_states.append([state_h2])

#   # Third encoder layer of the rnn
#   if num_encoders > 2:
#     encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     enc_layers.append(encoder)
#     enc_out, state_h3 = encoder(enc_out)
#     enc_states.append([state_h3])

  #==============================================#  
  #============== Decoder part ==================#
  #==============================================#

  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _ = decoder(dec_out, initial_state = enc_states[i])


#   # First decoder layer of the rnn
#   decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#   dec_layers.append(decoder)
#   dec_out, _ = decoder(dec_out, initial_state = enc_states[0])

#   # Second decoder layer of the rnn
#   if num_decoders > 1:
#     decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     dec_layers.append(decoder)
#     dec_out, _ = decoder(dec_out, initial_state = enc_states[1])

#   # Third decoder layer of the rnn
#   if num_decoders > 2:
#     decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     dec_layers.append(decoder)
#     dec_out, _ = decoder(dec_out, initial_state = enc_states[2])
    
  dec_dense = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [19]:
def lstm( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder


  #=============================================#
  #============== Encoder part =================#
  #=============================================#
  
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []

  for i in range(num_encoders):
        encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h, state_c = encoder(enc_out)
        enc_states.append([state_h, state_c])


  
#   encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#   enc_layers.append(encoder)
#   enc_out, state_h, state_c = encoder(enc_out)
#   enc_states.append([state_h, state_c])
  
#   if num_encoders > 1:
#     encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     enc_layers.append(encoder)
#     enc_out, state_h2, state_c2 = encoder(enc_out)
#     enc_states.append([state_h2, state_c2])
  
#   if num_encoders > 2:
#     encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     enc_layers.append(encoder)
#     enc_out, state_h3, state_c3 = encoder(enc_out)
#     enc_states.append([state_h3, state_c3])
    

   

  #==============================================#  
  #============== Decoder part ==================#
  #==============================================#


  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _, _ = decoder(dec_out, initial_state = enc_states[i])


#   decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#   dec_layers.append(decoder)
#   dec_out, _, _ = decoder(dec_out, initial_state = enc_states[0])

#   if num_decoders >1:
#     decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     dec_layers.append(decoder)
#     dec_out, _, _ = decoder(dec_out, initial_state = enc_states[1])
  
#   if num_decoders >2:
#     decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     dec_layers.append(decoder)
#     dec_out, _, _ = decoder(dec_out, initial_state = enc_states[2])

  
  
  dec_dense = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [20]:
def gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
 
  #=============================================#
  #============== Encoder part =================#
  #=============================================#
  

  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []

  for i in range(num_encoders):
        encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h = encoder(enc_out)
        enc_states.append([state_h])
  
#   # First encoder layer of the rnn
#   encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#   enc_layers.append(encoder)
#   enc_out, state_h = encoder(enc_out)
#   enc_states.append([state_h])
  
#   # Second encoder layer of the rnn
#   if num_encoders > 1:
#     encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     enc_layers.append(encoder)
#     enc_out, state_h2 = encoder(enc_out)
#     enc_states.append([state_h2])

#   # Third encoder layer of the rnn
#   if num_encoders > 2:
#     encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     enc_layers.append(encoder)
#     enc_out, state_h3 = encoder(enc_out)
#     enc_states.append([state_h3])
  #==============================================#  
  #============== Decoder part ==================#
  #==============================================#


  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _ = decoder(dec_out, initial_state = enc_states[i])


#   # First decoder layer of the rnn
#   decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#   dec_layers.append(decoder)
#   dec_out, _ = decoder(dec_out, initial_state = enc_states[0])

#   # Second decoder layer of the rnn
#   if num_decoders > 1:
#     decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     dec_layers.append(decoder)
#     dec_out, _ = decoder(dec_out, initial_state = enc_states[1])

#   # Third decoder layer of the rnn
#   if num_decoders > 2:
#     decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
#     dec_layers.append(decoder)
#     dec_out, _ = decoder(dec_out, initial_state = enc_states[2])
    
  dec_dense =TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [37]:
def build_model(num_encoders, num_decoders, cell, embed_size, dropout, hidden_layer_size):


  if cell == "RNN":
    model, enc_layers, dec_layers=rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "LSTM":
    model, enc_layers, dec_layers=lstm(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "GRU":
    model, enc_layers, dec_layers=gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  


In [22]:

batch_size = 128        
epochs = 7             
input_embedding_size = 512
hidden_layer_size = 256
num_layers = 3
num_encoder_layers = num_layers
num_decoder_layers = num_layers
dropout = 0.2
cell_type = 'GRU'

config = {
    "num_encoders" : 3,
    "cell": "GRU",
    "embed_size": 512,
    "dropout": 0.2,
    "num_decoders" : 3,
    "hidden_layer_size":256
}

# TRAIN
model, encoder_layers, decoder_layers = transliteration(config)

# COMPILE
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# FIT
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    shuffle = True,
    # validation_data= ([encoder_val_input_data, decoder_val_input_data], decoder_val_target_data)
)

Epoch 1/7
458/458 [==============================] - 23s 24ms/step - loss: 1.1296 - accuracy: 0.1745
Epoch 2/7
458/458 [==============================] - 11s 24ms/step - loss: 0.5457 - accuracy: 0.3280
Epoch 3/7
458/458 [==============================] - 11s 24ms/step - loss: 0.3564 - accuracy: 0.3792
Epoch 4/7
458/458 [==============================] - 11s 24ms/step - loss: 0.2937 - accuracy: 0.3951
Epoch 5/7
458/458 [==============================] - 11s 24ms/step - loss: 0.2595 - accuracy: 0.4028
Epoch 6/7
458/458 [==============================] - 11s 24ms/step - loss: 0.2356 - accuracy: 0.4080
Epoch 7/7
458/458 [==============================] - 11s 24ms/step - loss: 0.2186 - accuracy: 0.4111


In [26]:
def inferencing(model, num_encoders, num_decoders, enc_layers, dec_layers, cell, hidden_layer_size):

    enc_in = model.input[0]
    EmbeddingLayer = model.layers[2]    
    enc_out = EmbeddingLayer(enc_in)

    enc_states = []
    if cell == 'RNN':
        for i in range(num_encoders):
            enc_out, state_h = enc_layers[i](enc_out)
            enc_states += [state_h] 
    elif cell == 'LSTM':
        for i in range(num_encoders):
            enc_out, state_h, state_c = enc_layers[i](enc_out)
            enc_states += [state_h, state_c]   
    elif cell == 'GRU':
        for i in range(num_encoders):
            enc_out, state_h = enc_layers[i](enc_out)
            enc_states += [state_h] 

    encoder = Model(enc_in, enc_states + [enc_out])


    dec_in = model.input[1]    
    EmbeddingLayer = model.layers[3] 
    dec_out = EmbeddingLayer(dec_in)

    dec_states = []
    dec_initial_states = []
    
    if cell == 'RNN' :
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size,))]
            dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
            dec_states += [state_h]
    elif cell == "LSTM":
        j=0
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size, )) , Input(shape=(hidden_layer_size, ))]
            dec_out, state_h, state_c = dec_layers[i](dec_out, initial_state=dec_initial_states[i+j:i+j+2])
            dec_states += [state_h , state_c]
            j += 1
    elif cell == "GRU":
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size,))]
            dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
            dec_states += [state_h]

    dec_dense = model.layers[4 + 2*num_encoders]
    dec_out = dec_dense(dec_out)
    decoder = Model([dec_in] + dec_initial_states, [dec_out] + dec_states)

    return encoder, decoder 

In [27]:
def decode_sequence(input_seq, encoder, decoder):

    states_value = encoder.predict(input_seq)
    states_value = states_value[:-1]

    target_seq = np.zeros((1, 1)) 
    target_seq[0, 0] = target_token_index["\t"]

    #print(input_seq)

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens = decoder.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0][0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = output_tokens[1:]
        
    return decoded_sentence

In [28]:
encoder_model, decoder_model = inferencing(model, num_encoder_layers, num_decoder_layers, encoder_layers, decoder_layers, cell_type, hidden_layer_size)
correct = 0
n = val_target.shape[0]
for i in range(20):
    inputs = encoder_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model, decoder_model)
    print("-")
    print("Input sentence:", input[i])
    print("Decoded sentence:", output)

-
Input sentence: amkita
Decoded sentence: అంకిత

-
Input sentence: ankita
Decoded sentence: అంకిత

-
Input sentence: ankitha
Decoded sentence: అంకిత

-
Input sentence: ankitam
Decoded sentence: అంకితరం

-
Input sentence: ankitham
Decoded sentence: అంకితరం

-
Input sentence: ankitabaavam
Decoded sentence: అంగికాషాంగా

-
Input sentence: ankithabhavam
Decoded sentence: అంకితరంచా

-
Input sentence: ankatamicchaadu
Decoded sentence: అంకితమిచచ్చి

-
Input sentence: ankitamicchadu
Decoded sentence: అంకితరించాడు

-
Input sentence: ankitamichhaadu
Decoded sentence: అంకితరించాడు

-
Input sentence: ankithamicchaadu
Decoded sentence: అంకితరించాడు

-
Input sentence: ankithamichhaadu
Decoded sentence: అంకితరించాడు

-
Input sentence: amkithamichaaru
Decoded sentence: అంకితమిచచ్చి

-
Input sentence: ankithamichaaru
Decoded sentence: అంకితమిచచ్చి

-
Input sentence: ankithamicharu
Decoded sentence: అంకితరించారు

-
Input sentence: ankithamaina
Decoded sentence: అంకితమించాయి

-
Input sentence: ankusham
D

In [ ]:
def train_model(config, encoder_input, decoder_input, decoder_target, encoder_val_input, decoder_val_input, decoder_val_target ):

    batch_size = config['batch_size']
    cell = config['cell']
    embed_size = config['embed_size']
    dropout = config['dropout']
    num_encoders = config['num_encoders']
    num_decoders = config['num_decoders']
    hidden_layer_size = config['hidden_layer_size']
    epochs = config['epochs']
    optimizer = config['optimizer']


    model, encoder_layers, decoder_layers = build_model(num_encoders, num_decoders, cell, embed_size, dropout, hidden_layer_size)

    model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

    model.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size = batch_size,
    epochs = epochs,
    shuffle = True,
    validation_data= ([encoder_val_input, decoder_val_input], decoder_val_target)
    )

    return model





    

In [31]:
# Enter the project and the entity name for wandb sweeps
project_name = "Assignment 3"
entity_name = "safikhan"


In [40]:
def train():
  default_config={
      "cell": "RNN",
      "embed_size":256,
      "dropout":0.2,
      "num_layers": 1,
      "hidden_layer_size":128,
      "epochs": 1,
      "batch_size": 64,
      "optimizer": 'adam'
  }

  wandb.init(config=default_config, project=project_name, entity=entity_name)
  config = wandb.config
  wandb.run.name = str(config.cell)\
      +"_"+str(config.embed_size)\
      +"_"+str(config.dropout)\
      +"_"+str(config.num_encoders)\
      +"_"+str(config.num_decoders)\
      +"_"+str(config.hidden_layer_size)\
      +"_"+str(config.epochs)\
      +"_"+str(config.batch_size)

  #wandb.run.save()
  
  
  batch_size = config['batch_size']
  cell = config['cell']
  embed_size = config['embed_size']
  dropout = config['dropout']
  num_encoders = config['num_layers']
  num_decoders = config['num_layers']
  hidden_layer_size = config['hidden_layer_size']
  epochs = config['epochs']
  optimizer = config['optimizer']


  model, encoder_layers, decoder_layers = build_model(num_encoders, num_decoders, cell, embed_size, dropout, hidden_layer_size)

  
  model.compile(
      optimizer = optimizer,
      loss = tf.keras.losses.CategoricalCrossentropy(name='loss'),
      metrics = [tf.keras.metrics.CategoricalAccuracy(name='acc')]
  )
  
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size = batch_size,
      epochs = epochs,
      shuffle = True,
      validation_data= ([encoder_val_input_data, decoder_val_input_data], decoder_val_target_data),
      callbacks=[WandbCallback()]
  )

  wandb.run.finish()

  encoder_model, decoder_model = inferencing(model, num_encoder_layers, num_decoder_layers, encoder_layers, decoder_layers, cell_type, hidden_layer_size)
  correct = 0
  n = val_input.shape[0]
  for i in range(n):
    inputs = encoder_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model, decoder_model)
    # print("-")
    # print("Input sentence:", input[i])
    # print("Decoded sentence:", output)

    if output.strip() == val_target[i].strip():
            correct += 1

    print("The accuracy based on full matching is: ", correct/n)

    wandb.log({'Full_Val_Accuract' : correct/n})



  

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fab43751cd0>> (for pre_run_cell):


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fab43751cd0>> (for post_run_cell):


Exception: ignored

In [34]:
hyperparameters = {

          "cell":{
              "values":["RNN","GRU","LSTM"]
              },

          "embed_size":{
              "values":[16,32,64,256]
              },

          "hidden_layer_size":{
              "values":[16,32,64,256]
              },

          "num_layers":{
              "values":[1,2,3]
              },

          "dropout":{
              "values":[0.2,0.3,0.4]
              },

          "epochs":{
              "values":[5,10,15,20]
              },

          "batch_size":{
              "values":[32,64]
              },

          "optimizer":{
              "values":["adam", "rsmprop", "nadam"]
              }
      }

In [35]:
def sweeper(project_name,entity_name):
  sweep_config={

      "method": "bayes",
      "metric": {
          "name": "val_acc", 
          "goal": "maximize"
          },
      "parameters":hyperparameters
  }

  sweep_id=wandb.sweep(sweep_config, project=project_name, entity=entity_name)
  wandb.agent(sweep_id,train)


In [38]:
# Running the sweeps for hyperparameter tuning
sweeper(project_name,entity_name)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 79hbhj80
Sweep URL: https://wandb.ai/safikhan/Assignment%203/sweeps/79hbhj80


wandb: Agent Starting Run: kcvw42tv with config:
wandb: 	batch_size: 64
wandb: 	cell: LSTM
wandb: 	dropout: 0.3
wandb: 	embed_size: 32
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 32
wandb: 	num_decoders: 2
wandb: 	num_encoders: 1
wandb: 	optimizer: rsmprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


wandb: ERROR Run kcvw42tv errored: IndexError('list index out of range')
wandb: Agent Starting Run: b97arbbc with config:
wandb: 	batch_size: 32
wandb: 	cell: GRU
wandb: 	dropout: 0.2
wandb: 	embed_size: 256
wandb: 	epochs: 15
wandb: 	hidden_layer_size: 16
wandb: 	num_decoders: 2
wandb: 	num_encoders: 2
wandb: 	optimizer: nadam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/15
1159/1830 [==================>...........] - ETA: 8s - loss: 1.4805 - acc: 0.0907

wandb: Ctrl + C detected. Stopping sweep.
